## Classification of Objects in selected bricks into ELG, LRG, QSO

In [35]:
import sys
import numpy as np
from astropy import constants as const
from astropy import units as u
from astropy.io import fits
from astropy.io import ascii
import random
import matplotlib.pyplot as plt
import wget
import seaborn as sns
import random
import os
import warnings
import time
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
import pickle

warnings.filterwarnings("ignore")
random.seed(666)
#import healpy

In [36]:
from platform import python_version
print(python_version())

3.8.8


### Functions to deal with maskbits


In [45]:
def get_imaging_maskbits(bitnamelist=None):
    """Return MASKBITS names and bits from the Legacy Surveys.

    Parameters
    ----------
    bitnamelist : :class:`list`, optional, defaults to ``None``
        If not ``None``, return the bit values corresponding to the
        passed names. Otherwise, return the full MASKBITS dictionary.

    Returns
    -------
    :class:`list` or `dict`
        A list of the MASKBITS values if `bitnamelist` is passed,
        otherwise the full MASKBITS dictionary of names-to-values.

    Notes
    -----
    - For the definitions of the mask bits, see, e.g.,
      https://www.legacysurvey.org/dr8/bitmasks/#maskbits
    """
    bitdict = {"BRIGHT": 1, "ALLMASK_G": 5, "ALLMASK_R": 6, "ALLMASK_Z": 7,
               "BAILOUT": 10, "MEDIUM": 11, "GALAXY": 12, "CLUSTER": 13}

    # ADM look up the bit value for each passed bit name.
    if bitnamelist is not None:
        return [bitdict[bitname] for bitname in bitnamelist]

    return bitdict

def get_default_maskbits(bgs=False, mws=False):
    """Return the names of the default MASKBITS for targets.

    Parameters
    ----------
    bgs : :class:`bool`, defaults to ``False``.
        If ``True`` load the "default" scheme for Bright Galaxy Survey
        targets. Otherwise, load the default for other target classes.
    mws : :class:`bool`, defaults to ``False``.
        If ``True`` load the "default" scheme for Milky Way Survey
        targets. Otherwise, load the default for other target classes.

    Returns
    -------
    :class:`list`
        A list of the default MASKBITS names for targets.

    Notes
    -----
    - Only one of `bgs` or `mws` can be ``True``.
    """
    if bgs and mws:
        msg = "Only one of bgs or mws can be passed as True"
        #Adapting to print messages
        print(msg)
        #log.critical(msg)
        raise ValueError(msg)
    if bgs:
        return ["BRIGHT", "CLUSTER"]
    if mws:
        return ["BRIGHT", "GALAXY"]

    return ["BRIGHT", "GALAXY", "CLUSTER"]

def imaging_mask(maskbits, bitnamelist=get_default_maskbits(),
                 bgsmask=False, mwsmask=False):
    """Apply the 'geometric' masks from the Legacy Surveys imaging.

    Parameters
    ----------
    maskbits : :class:`~numpy.ndarray` or ``None``
        General array of `Legacy Surveys mask`_ bits.
    bitnamelist : :class:`list`, defaults to func:`get_default_maskbits()`
        List of Legacy Surveys mask bits to set to ``False``.
    bgsmask : :class:`bool`, defaults to ``False``.
        Load the "default" scheme for Bright Galaxy Survey targets.
        Overrides `bitnamelist`.
    bgsmask : :class:`bool`, defaults to ``False``.
        Load the "default" scheme for Milky Way Survey targets.
        Overrides `bitnamelist`.

    Returns
    -------
    :class:`~numpy.ndarray`
        A boolean array that is the same length as `maskbits` that
        contains ``False`` where any bits in `bitnamelist` are set.

    Notes
    -----
    - Only one of `bgsmask` or `mwsmask` can be ``True``.
    """
    # ADM default for the BGS or MWS..
    if bgsmask or mwsmask:
        bitnamelist = get_default_maskbits(bgs=bgsmask, mws=mwsmask)

    # ADM get the bit values for the passed (or default) bit names.
    bits = get_imaging_maskbits(bitnamelist)

    # ADM Create array of True and set to False where a mask bit is set.
    mb = np.ones_like(maskbits, dtype='?')
    for bit in bits:
        mb &= ((maskbits & 2**bit) == 0)

    return mb


### Defining colour cuts - LRG

In [44]:
def isLRG(gflux=None, rflux=None, zflux=None, w1flux=None, w2flux=None,
          zfiberflux=None, rfluxivar=None, zfluxivar=None, w1fluxivar=None,
          gaiagmag=None, gnobs=None, rnobs=None, znobs=None, maskbits=None,
          zfibertotflux=None, primary=None, south=True):
    """
    Parameters
    ----------
    south: boolean, defaults to ``True``
        Use cuts appropriate to the Northern imaging surveys (BASS/MzLS)
        if ``south=False``, otherwise use cuts appropriate to the
        Southern imaging survey (DECaLS).

    Returns
    -------
    :class:`array_like`
        ``True`` if and only if the object is an LRG target.

    Notes
    -----
    - Current version (05/07/21) is version 260 on `the wiki`_.
    - See :func:`~desitarget.cuts.set_target_bits` for other parameters.
    """
    # ADM LRG targets.
    if primary is None:
        primary = np.ones_like(rflux, dtype='?')
    lrg_quality = primary.copy()

    # ADM basic quality cuts.
    lrg_quality &= notinLRG_mask(
        primary=primary, rflux=rflux, zflux=zflux, w1flux=w1flux,
        zfiberflux=zfiberflux, gnobs=gnobs, rnobs=rnobs, znobs=znobs,
        rfluxivar=rfluxivar, zfluxivar=zfluxivar, w1fluxivar=w1fluxivar,
        gaiagmag=gaiagmag, maskbits=maskbits, zfibertotflux=zfibertotflux
    )

    # ADM color-based selection of LRGs.
    lrg = isLRG_colors(
        gflux=gflux, rflux=rflux, zflux=zflux, w1flux=w1flux,
        zfiberflux=zfiberflux, south=south, primary=primary
    )

    lrg &= lrg_quality

    return lrg

def notinLRG_mask(primary=None, rflux=None, zflux=None, w1flux=None,
                  zfiberflux=None, gnobs=None, rnobs=None, znobs=None,
                  rfluxivar=None, zfluxivar=None, w1fluxivar=None,
                  gaiagmag=None, maskbits=None, zfibertotflux=None):
    """See :func:`~desitarget.cuts.isLRG` for details.

    Returns
    -------
    :class:`array_like`
        ``True`` if and only if the object is NOT masked for poor quality.
    """
    if primary is None:
        primary = np.ones_like(rflux, dtype='?')
    lrg = primary.copy()

    # ADM to maintain backwards-compatibility with mocks.
    if zfiberflux is None:
        print('Setting zfiberflux to zflux!!!') ## Altered in my code to keep compatibilty
        #log.warning('Setting zfiberflux to zflux!!!')
        zfiberflux = zflux.copy()

    lrg &= (rfluxivar > 0) & (rflux > 0)   # ADM quality in r.
    lrg &= (zfluxivar > 0) & (zflux > 0) & (zfiberflux > 0)   # ADM quality in z.
    lrg &= (w1fluxivar > 0) & (w1flux > 0)  # ADM quality in W1.

    lrg &= (gaiagmag == 0) | (gaiagmag > 18)  # remove bright GAIA sources

    # ADM remove stars with zfibertot < 17.5 that are missing from GAIA.
    lrg &= zfibertotflux < 10**(-0.4*(17.5-22.5))

    # ADM observed in every band.
    lrg &= (gnobs > 0) & (rnobs > 0) & (znobs > 0)

    # ADM default mask bits from the Legacy Surveys not set.
    lrg &= imaging_mask(maskbits)

    return lrg

def isLRG_colors(gflux=None, rflux=None, zflux=None, w1flux=None,
                 zfiberflux=None, ggood=None,
                 w2flux=None, primary=None, south=True):
    """(see, e.g., :func:`~desitarget.cuts.isLRG`).

    Notes:
        - the `ggood` and `w2flux` inputs are an attempt to maintain
          backwards-compatibility with the mocks.
    """
    if primary is None:
        primary = np.ones_like(rflux, dtype='?')
    lrg = primary.copy()

    # ADM to maintain backwards-compatibility with mocks.
    if zfiberflux is None:
        print('Setting zfiberflux to zflux!!!') ## Altered in my code to keep compatibilty
        #log.warning('Setting zfiberflux to zflux!!!')
        zfiberflux = zflux.copy()

    gmag = 22.5 - 2.5 * np.log10(gflux.clip(1e-7))
    # ADM safe as these fluxes are set to > 0 in notinLRG_mask.
    rmag = 22.5 - 2.5 * np.log10(rflux.clip(1e-7))
    zmag = 22.5 - 2.5 * np.log10(zflux.clip(1e-7))
    w1mag = 22.5 - 2.5 * np.log10(w1flux.clip(1e-7))
    zfibermag = 22.5 - 2.5 * np.log10(zfiberflux.clip(1e-7))

    # Full SV3 selection
    if south:
        lrg &= zmag - w1mag > 0.8 * (rmag - zmag) - 0.6  # non-stellar cut
        lrg &= zfibermag < 21.6                   # faint limit
        lrg &= (gmag - w1mag > 2.9) | (rmag - w1mag > 1.8)  # low-z cuts
        lrg &= (
            ((rmag - w1mag > (w1mag - 17.14) * 1.8)
             & (rmag - w1mag > (w1mag - 16.33) * 1.))
            | (rmag - w1mag > 3.3)
        )  # double sliding cuts and high-z extension
    else:
        lrg &= zmag - w1mag > 0.8 * (rmag - zmag) - 0.6  # non-stellar cut
        lrg &= zfibermag < 21.61                   # faint limit
        lrg &= (gmag - w1mag > 2.97) | (rmag - w1mag > 1.8)  # low-z cuts
        lrg &= (
            ((rmag - w1mag > (w1mag - 17.13) * 1.83)
             & (rmag - w1mag > (w1mag - 16.31) * 1.))
            | (rmag - w1mag > 3.4)
        )  # double sliding cuts and high-z extension

    return lrg

### Defining Colour



In [37]:
def is_LRG_target(g,r,z,W1):
    if not (18.01 < z < 20.41):
        return False
    if not (0.75 < (r - z) < 2.45):
        return False
    if not (-0.6 < (z - W1) - 0.8*(r - z)):
        return False
    if not ((z - 17.18)/2 < (r - z) < (z - 15.11)/2):
        return False
    if not (((r - z) > 1.15) or ((g-r) > 1.65)):
        return False
    return True
    
def is_ELG_target(g,r,z):
    if not (21.0 < g < 23.45):
        return False
    if not (0.3 < (r - z) < 1.6):
        return False
    if not ((g-r) < 1.15*(r-z) - 0.15):
        return False
    if not ((g-r) < 1.6 - 1.2*(r-z)):
        return False
    return True

def is_QSO_target(g,r,z,maskbit):
    if r > 22.7:
        return False
    if r < 17.5:
        return False
    if not g - r < 1.3:
        return False
    if not (-0.4 < r - z < 1.1):
        return False
    return True
        

### Getting bricks data

In [42]:
hdulistBricksSummary = fits.open('../bricks_data/survey-bricks-dr9-south.fits')
dataSummary = hdulistBricksSummary[1].data
bricknameSummary = dataSummary.field('brickname')
brickidSummary = dataSummary.field('brickid')
brick_galaxy_info = np.array([[220271,34,10,177]])
# Todo: Get new reference brick after I finished the appropriate classification

## Classification Loop
#### Steps:
##### 1. Download a randomly sampled brick
##### 2. Calculate extinction corrected magnitudes ( m=22.5−2.5log10(flux))
##### 3. Apply colour cuts
##### 4. Delete brick data

In [39]:
# Asserting that numerical brickId is also unique and can be used as more storage efficient index to a brick
print(len(bricknameSummary))
print(len(brickidSummary))
print(len(np.unique(brickidSummary)))

253658
253658
253658


In [43]:
start = time.time()

for brick in range(1):
    
    #Sampling a random brick and preparing the data
    randomint = random.randint(0, len(bricknameSummary))
    brickname = bricknameSummary[randomint] 
    brickid = brickidSummary[randomint] 
    
    folder = brickname[:3]
    url = f'https://portal.nersc.gov/cfs/cosmo/data/legacysurvey/dr9/south/tractor/{folder}/tractor-{brickname}.fits'
    #wget.download(url, '../bricks_data/tractor/')
    wget.download(url, '/Volumes/Astrostick/bricks_data/')

    #hdulistSingleBrick = fits.open(f'../bricks_data/tractor/tractor-{brickname}.fits')

    hdulistSingleBrick = fits.open(f'/Volumes/Astrostick/bricks_data/tractor-{brickname}.fits')
    data = hdulistSingleBrick[1].data
    
    #fluxes and magnitude
    flux_g = data.field('flux_g')
    flux_r = data.field('flux_r')
    flux_z = data.field('flux_z')
    flux_w1 = data.field('flux_w1')
    flux_w2 = data.field('flux_w2')
    flux_w3 = data.field('flux_w3')
    flux_w4 = data.field('flux_w4')

    mw_transmission_g = data.field('mw_transmission_g')
    mw_transmission_r = data.field('mw_transmission_r')
    mw_transmission_z = data.field('mw_transmission_z')
    mw_transmission_w1 = data.field('mw_transmission_w1')
    mw_transmission_w2 = data.field('mw_transmission_w2')
    mw_transmission_w3 = data.field('mw_transmission_w3')
    mw_transmission_w4  = data.field('mw_transmission_w4')

    #correcting for extinction ---> divide by the transmission
    flux_g_corrected = flux_g / mw_transmission_g
    flux_r_corrected = flux_r / mw_transmission_r
    flux_z_corrected = flux_z / mw_transmission_z
    flux_w1_corrected = flux_w1 / mw_transmission_w1
    flux_w2_corrected = flux_w2 / mw_transmission_w2
    flux_w3_corrected = flux_w3 / mw_transmission_w3
    flux_w4_corrected = flux_w4 / mw_transmission_w4



    mag_g = 22.5-2.5*np.log10(flux_g_corrected)
    mag_r = 22.5-2.5*np.log10(flux_r_corrected)
    mag_z = 22.5-2.5*np.log10(flux_z_corrected)
    mag_w1 = 22.5-2.5*np.log10(flux_w1_corrected)
    mag_w2 = 22.5-2.5*np.log10(flux_w2_corrected)
    mag_w3 = 22.5-2.5*np.log10(flux_w3_corrected)
    mag_w4 = 22.5-2.5*np.log10(flux_w4_corrected)

    #Retrieving the maskbits for quasar detection

    maskbits = data.field('maskbits')

    target_label_array = np.zeros(len(mag_g))
    for i in range(len(mag_g)):
        if is_LRG_target(mag_g[i], mag_r[i], mag_z[i], mag_w1[i]):
            target_label_array[i] = 1
            continue
        if is_ELG_target(mag_g[i], mag_r[i], mag_z[i]):
            target_label_array[i] = 2
            continue
        if is_QSO_target(mag_g[i], mag_r[i], mag_z[i], maskbits[i]):
            target_label_array[i] = 3
            continue
            
    lrg = (target_label_array == 1).sum()
    elg = (target_label_array == 2).sum()
    qso = (target_label_array == 3).sum()
    
    summary_stats = np.array([[brickid,lrg,elg,qso]])

    brick_galaxy_info = np.append(brick_galaxy_info, summary_stats, axis=0)
    
    #os.remove(f'../bricks_data/tractor/tractor-{brickname}.fits')
    #os.remove(f'/Volumes/Astrostick/bricks_data/tractor-{brickname}.fits')

    if (brick % 50) == 0:
        print("Bricks finished:", brick, " of 1000")

print(brick_galaxy_info)

print("Time taken: ", time.time()- start)

Bricks finished: 0  of 1000
[[220271     34     10    177]
 [492321     22    162   1029]]
Time taken:  5.350919723510742


In [ ]:
print(len(brick_galaxy_info))


### Beginning preliminary analysis

In [ ]:
sample = brick_galaxy_info
nexp_gSummary = dataSummary.field('nexp_g')
nexp_rSummary = dataSummary.field('nexp_r')
nexp_zSummary = dataSummary.field('nexp_z')



In [ ]:
print(sample.shape)

In [ ]:
sample = np.c_[sample, np.zeros(len(sample))]
print(sample.shape)
print(sample)

In [ ]:
print(sample[:,0])
ind = 0
for j in range(len(brickidSummary)):
    if sample[0,0] == brickidSummary[j]:
        ind = j
print(ind)
print(nexp_gSummary[ind])
print(nexp_rSummary[ind])
print(nexp_zSummary[ind])


In [ ]:
#Exporting the data
with open('sample_brick_catalogue.pickle', 'wb') as f:
    pickle.dump(sample, f)

In [ ]:

sampled = sample
for i in range(len(sampled)):
    index = 0
    for j in range(len(brickidSummary)):
        if sampled[i,0] == brickidSummary[j]:
            index = j
    sampled[i, 4] = nexp_gSummary[index]
    sampled[i, 5] = nexp_rSummary[index]
    sampled[i, 6] = nexp_zSummary[index]

In [ ]:
elg = sampled[:, 1].reshape(-1, 1)
lrg = sampled[:, 2].reshape(-1, 1)
olg = sampled[:, 3].reshape(-1, 1)
exposure_g = sampled[:, 4].reshape(-1, 1)
exposure_r = sampled[:, 5].reshape(-1, 1)
exposure_z = sampled[:, 6].reshape(-1, 1)

In [ ]:
reg_g_elg = LinearRegression().fit(exposure_g, elg)
print(reg_g_elg.score(exposure_g, elg))

In [ ]:
elg = sampled[:, 1]
exposure_g = sampled[:, 4].reshape(-1,1)
print(elg.shape)
print(exposure_g.shape)
ols_g_elg = sm.OLS(elg,exposure_g).fit()
print(ols_g_elg.summary())